# <b>CS156 (Introduction to AI), Spring 2022</b>
# <u><b>Final term project</b></u>

### Roster Name: Andy
### Student ID: 014222676
### Email address: andy.v.luong@sjsu.edu

##  <u> Project description/introduction text (the background information) </u>

Add the description of this project. Describe the background of the problem your project is trying to solve. Describe the ML problem you are solving below. 

##  <u> Machine learning algorithm selected for this project </u>

Name and add short description of the algorithm used for this project. You can re-use this text in your final poster submission.
Decision Tree & Random Forest

##  <u> Dataset source </u>

List the source from where the dataset for this project was obtained.
https://www.kaggle.com/competitions/titanic/overview

##  <u> References and sources </u>

List all your references and sources here.
This includes all sites/discussion boards/blogs/posts/etc. where you grabbed some code examples.
IF/ELSE All Values = https://stackoverflow.com/questions/44991438/lambda-including-if-elif-else
Map = https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html
PLT & SNS = #https://machinelearningknowledge.ai/seaborn-heatmap-using-sns-heatmap-with-examples-for-beginners/

##  <u>Solution</u>

#### Load libraries and set random number generator seed

In [1]:
import numpy as np
import pandas as pd
from sklearn import tree #tree
from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
np.random.seed(42)

#### Code the solution

###Making Titanic Dataset

In [15]:
# Loading the 
try:
    from google.colab import files
    uploaded = files.upload()

    import io
    train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
    uploaded2 = files.upload()
    import io
    test = pd.read_csv(io.BytesIO(uploaded2['test.csv']))
except:
    train = pd.read_csv('train.csv')
    test =  pd.read_csv('test.csv')

In [7]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
test.head() #test does not know who survives

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


###Add Columns, Fix Nulls, Remove Unnecessary Variables, & Plot Seaborne

In [9]:
train2 = train.copy()
test2 = test.copy()

In [10]:
train = train2.copy()
test = test2.copy()

In [11]:
#Adds a column where...
#value is 1 if there is a cabin, 0 otherwise
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
#it combines SibSp and Parch to find family size
train['Family_Size'] = train['SibSp'] + train['Parch'] + 1
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Has_Cabin,Family_Size
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0,4
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,1


In [12]:
#Replaces Null Values in Embarked, Fare, & Age With the Average Value
train['Embarked'] = train['Embarked'].fillna('S')
train['Fare'] = train['Fare'].fillna(train['Fare'].median())
train['Age'] = train['Age'].fillna(train['Age'].median())
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Has_Cabin,Family_Size
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,NaN,S,0,4
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,1


In [13]:
# Map Sex
if (train['Sex'][0] != 0) and (train['Sex'][0] != 1):
  train['Sex'] = train['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# Map Embarked
if (train['Embarked'][0] != 0) and (train['Embarked'][0] != 1) and (train['Embarked'][0] != 2):
  train['Embarked'] = train['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
# Map Fare
fare_min = train['Fare'].min()
fare_q1 = train['Fare'].quantile(.25)
fare_median = train['Fare'].median()
fare_q3 = train['Fare'].quantile(.75)
fare_max = train['Fare'].max()
fare_min, fare_q1, fare_median, fare_q3, fare_max

train.loc[(train['Fare'] >= fare_min) & (train['Fare'] <= fare_q1), 'Fare'] = 0
train.loc[(train['Fare'] > fare_q1) & (train['Fare'] < fare_median), 'Fare'] = 1
train.loc[(train['Fare'] > fare_median) & (train['Fare'] < fare_q3), 'Fare'] = 2
train.loc[(train['Fare'] >= fare_q3) & (train['Fare'] <= fare_max), 'Fare'] = 3
    
# Map Age
age_min = train['Age'].min()
age_q1 = train['Age'].quantile(.25)
age_median = train['Age'].median()
age_q3 = train['Age'].quantile(.75)
age_max = train['Age'].max()
age_min, age_q1, age_median, age_q3, age_max

train.loc[(train['Age'] >= age_min) & (train['Age'] <= age_q1), 'Age'] = 0
train.loc[(train['Age'] > age_q1) & (train['Age'] < age_median), 'Age'] = 1
train.loc[(train['Age'] > age_median) & (train['Age'] < age_q3), 'Age'] = 2
train.loc[(train['Age'] >= age_q3) & (train['Age'] <= age_max), 'Age'] = 3
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Has_Cabin,Family_Size
0,1,0,3,"Braund, Mr. Owen Harris",1,0.0,1,0,A/5 21171,0.0,NaN,0,0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,3.0,1,0,PC 17599,3.0,C85,1,1,2
2,3,1,3,"Heikkinen, Miss. Laina",0,1.0,0,0,STON/O2. 3101282,1.0,NaN,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,3.0,1,0,113803,3.0,C123,0,1,2
4,5,0,3,"Allen, Mr. William Henry",1,3.0,0,0,373450,1.0,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,1.0,0,0,211536,1.0,NaN,0,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",0,0.0,0,0,112053,2.0,B42,0,1,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,28.0,1,2,W./C. 6607,2.0,NaN,0,0,4
889,890,1,1,"Behr, Mr. Karl Howell",1,1.0,0,0,111369,2.0,C148,1,1,1


In [14]:
#Drop columns that are not needed in calculation
try:
  dropColumns = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
  train = train.drop(dropColumns, axis = 1)
  train
except:
  print("Already Dropped")

In [ ]:
plt.figure(figsize=(15,15))
plt.title('Seaborn Heatmap of Dependent Variables')
sns.heatmap(train.astype(float).corr(),linewidths=0.6, annot=True)

###Survival Rate For Each Variable

In [ ]:
#mean = survival rate, count = total people, sum = survivors
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).agg(['mean', 'count', 'sum'])

In [ ]:
#mean = survival rate, count = total people, sum = survivors
train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).agg(['mean', 'count', 'sum'])

In [ ]:
train[['Age', 'Survived']].groupby(['Age'], as_index=False).agg(['mean', 'count', 'sum'])

In [ ]:
train[['Age', 'Survived']].groupby(['Age'], as_index=False).agg(['mean', 'count', 'sum'])

In [ ]:
train[['Parch', 'Survived']].groupby(['Parch'], as_index=False).agg(['mean', 'count', 'sum'])

In [ ]:
train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).agg(['mean', 'count', 'sum'])

In [ ]:
train[['Has_Cabin', 'Survived']].groupby(['Has_Cabin'], as_index=False).agg(['mean', 'count', 'sum'])

In [4]:
train[['Family_Size', 'Survived']].groupby(['Family_Size'], as_index=False).agg(['mean', 'count', 'sum'])

NameError: name 'train' is not defined

###Tree

In [ ]:
cv = KFold(n_splits=10)            # Desired number of Cross Validation folds
accuracies = list()
max_attributes = len(list(test))
depth_range = range(1, max_attributes + 1)

# Testing max_depths from 1 to max attributes
# Uncomment prints for details about each Cross Validation pass
for depth in depth_range:
    fold_accuracy = []
    tree_model = tree.DecisionTreeClassifier(max_depth = depth)
    # print("Current max depth: ", depth, "\n")
    for train_fold, valid_fold in cv.split(train):
        f_train = train.loc[train_fold] # Extract train data with cv indices
        f_valid = train.loc[valid_fold] # Extract valid data with cv indices

        model = tree_model.fit(X = f_train.drop(['Survived'], axis=1), 
                               y = f_train["Survived"]) # We fit the model with the fold train data
        valid_acc = model.score(X = f_valid.drop(['Survived'], axis=1), 
                                y = f_valid["Survived"])# We calculate accuracy with the fold validation data
        fold_accuracy.append(valid_acc)

    avg = sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)
    # print("Accuracy per fold: ", fold_accuracy, "\n")
    # print("Average accuracy: ", avg)
    # print("\n")
    
# Just to show results conveniently
df = pd.DataFrame({"Max Depth": depth_range, "Average Accuracy": accuracies})
df = df[["Max Depth", "Average Accuracy"]]
print(df.to_string(index=False))

In [ ]:
#Do the same for test
#Adds a column where...
#value is 1 if there is a cabin, 0 otherwise
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
#it combines SibSp and Parch to find family size
test['Family_Size'] = test['SibSp'] + test['Parch'] + 1
#Replaces Null Values in Embarked, Fare, & Age With the Average Value
test['Embarked'] = test['Embarked'].fillna('S')
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
test
#Replaces Null Values in Embarked, Fare, & Age With the Average Value
test['Embarked'] = test['Embarked'].fillna('S')
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
test
# Map Sex
if (test['Sex'][0] != 0) and (test['Sex'][0] != 1):
 test['Sex'] = test['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
 
# Map Embarked
if (test['Embarked'][0] != 0) and (test['Embarked'][0] != 1) and (test['Embarked'][0] != 2):
 test['Embarked'] = test['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
  
# Map Fare
fare_min = test['Fare'].min()
fare_q1 = test['Fare'].quantile(.25)
fare_median = test['Fare'].median()
fare_q3 = test['Fare'].quantile(.75)
fare_max = test['Fare'].max()
fare_min, fare_q1, fare_median, fare_q3, fare_max
 
test.loc[(test['Fare'] >= fare_min) & (test['Fare'] <= fare_q1), 'Fare'] = 0
test.loc[(test['Fare'] > fare_q1) & (test['Fare'] < fare_median), 'Fare'] = 1
test.loc[(test['Fare'] > fare_median) & (test['Fare'] < fare_q3), 'Fare'] = 2
test.loc[(test['Fare'] >= fare_q3) & (test['Fare'] <= fare_max), 'Fare'] = 3
  
# Map Age
age_min = test['Age'].min()
age_q1 = test['Age'].quantile(.25)
age_median = test['Age'].median()
age_q3 = test['Age'].quantile(.75)
age_max = test['Age'].max()
age_min, age_q1, age_median, age_q3, age_max
 
test.loc[(test['Age'] >= age_min) & (test['Age'] <= age_q1), 'Age'] = 0
test.loc[(test['Age'] > age_q1) & (test['Age'] < age_median), 'Age'] = 1
test.loc[(test['Age'] > age_median) & (test['Age'] < age_q3), 'Age'] = 2
test.loc[(test['Age'] >= age_q3) & (test['Age'] <= age_max), 'Age'] = 3
test
#Drop columns that are not needed in calculation
try:
 dropColumns = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
 test = test.drop(dropColumns, axis = 1)
 test
except:
 print("Already Dropped")

In [ ]:
# Create Numpy arrays of train, test and target (Survived) dataframes to feed into our models
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
y_train = train['Survived']
x_train = train.drop(['Survived'], axis=1).values 
x_test = test.values

# Create Decision Tree with max_depth = 3
decision_tree = tree.DecisionTreeClassifier(max_depth = 3)
decision_tree.fit(x_train, y_train)

# Predicting results for test dataset
PassengerId = test2['PassengerId']
y_pred = decision_tree.predict(x_test)
submission = pd.DataFrame({
        "PassengerId": PassengerId,
        "Survived": y_pred
    })
submission.to_csv('submission.csv', index=False)

# Export our trained model as a .dot file
with open("tree1.dot", 'w') as f:
     f = tree.export_graphviz(decision_tree,
                              out_file=f,
                              max_depth = 3,
                              impurity = True,
                              feature_names = list(train.drop(['Survived'], axis=1)),
                              class_names = ['Died', 'Survived'],
                              rounded = True,
                              filled= True )
        
#Convert .dot to .png to allow display in web notebook
check_call(['dot','-Tpng','tree1.dot','-o','tree1.png'])

# Annotating chart with PIL
img = Image.open("tree1.png")
draw = ImageDraw.Draw(img)
font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf', 26)
draw.text((10, 0), # Drawing offset (position)
          '"Title <= 1.5" corresponds to "Mr." title', # Text to draw
          (0,0,255), # RGB desired color
          font=font) # ImageFont object with desired font
img.save('sample-out.png')
PImage("sample-out.png")

# Code to check available fonts and respective paths
# import matplotlib.font_manager
# matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

In [ ]:
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
acc_decision_tree